In [ ]:
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [5]:
# 데이터 수집 도구로 selenium을 선택하였다.
# 왜 selenium인가? 빈 강의실을 확인하려면, 수강편람의 교수계획표를 클릭해서 확인해야 하는데,
# 수강편람에 있는 교수계획표가 수십개이므로, 일일이 클릭하고, 확인해야하는 수고로움이 있으므로,
# 이를 자동으로 해주는 selenium이 적합하다고 생각했다.

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

start_time = time.time()

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [6]:
# url : 부산대학교 학생지원시스템 수강편람(상세) 페이지. 대학, 학과, 전공을 선택해야한다.
url = "https://e-onestop.pusan.ac.kr/menu/class/C03/C03006?menuId=2000030306&rMenu=03"

# chrome 창을 띄움
driver.get(url)

# 수강 편람 페이지가 제대로 출력 될때까지 기다린다.
time.sleep(3)

In [7]:
# 내 정보를 수강편람 페이지에 작성하여, 내 학과의 정보를 검색하도록 하는 함수

def set_my_information(mycollege):
    collegelist = {"공과대학" : 5}  # 추후에 다른 대학도 추가할 수 있다.
    choose_college_button = driver.find_element(By.CSS_SELECTOR, '#opt_collCd > option:nth-child(' + str(collegelist[mycollege]) + ')')
    choose_college_button.click()
    time.sleep(1)

    choose_scholar_button = driver.find_element(By.CSS_SELECTOR, '#opt_deptCd > option:nth-child(13)')
    choose_scholar_button.click()
    time.sleep(1)

    choose_major_button = driver.find_element(By.CSS_SELECTOR, '#opt_mjrCd > option:nth-child(2)')
    choose_major_button.click()
    time.sleep(1)

    search_button = driver.find_element(By.ID, "btn_search")
    search_button.click()
    time.sleep(2)

In [8]:
# 내 과의 1, 2, 3, 4학년 모든 수업의 교수계획표에서, 강의 시간과 강의실 정보를 가져오는 함수

def get_class_information():
    classrooms = []

    for i in range(2, 64):
        try:
            l = driver.find_element(By.XPATH, '//*[@id="tbl_list"]/tbody/tr[' + str(i) + ']').text
            if '국문' in l or '영문' in l:
                processor_plans = driver.find_element(By.XPATH, '//*[@id="tbl_list"]/tbody/tr[' + str(i) + ']/td[13]/a')
                driver.execute_script("arguments[0].click();", processor_plans)

                time.sleep(1)
                classroom = driver.find_element(By.XPATH, '//*[@id="m2soft-crownix-text"]/div[15]').text
                classrooms.append(classroom)

                close_button = driver.find_element(By.CLASS_NAME, 'crownix-close-button')
                close_button.click()


        except:
            continue


    return classrooms

In [9]:
# 나는 '공과대학'에 속해 있으므로, 공대를 넣어주었다. 
# 추후에 모든 대학의 정보를 업데이트한다면, 자신의 대학, 학과를 넣을 수 있게 할 것이다.

set_my_information('공과대학')

# CLASSROOMS 변수에 내 정보(공과대학, 전기컴퓨터공학부, 정보컴퓨터공학전공, 전학년의 수업시간, 수업 강의실 정보)를 담아주었다.
CLASSROOMS = get_class_information()

# chrome 창 닫기
driver.close()

# selenium을 실행하는데 걸린 시간 출력
total_time = time.time() - start_time
print(total_time)


# selenium을 이용하여 데이터를 받아오는 시간은 약 3분정도 걸렸다.
# google colab에서는 selenium으로 chromedriver가 직접 돌아가는 모습을 확인은 못했지만,
# pycharm으로 제대로 교수계획표에서 데이터를 수집하는 것을 확인하였고, 신뢰할 만한 데이터이다.
# 교수계획표를 작성하지 않으신(?) 몇몇 교수님의 수업은 제대로 수집할 수 없었다.

193.36576795578003


In [12]:
# CLASSROOMS 정보에는 화, 목요일에 수업이 있는 정보가 한 줄에 저장되어 있으므로,
# final_classrooms 변수에 따로 따로 화, 목요일 정보를 저장하였다. 

final_classrooms = []
for c in CLASSROOMS:
  c = list(c.split(','))
  for a in c:
    final_classrooms.append(a)

In [13]:
# final_classrooms 정보를 살펴보면, 15:00(수업시간) 201-6xxx 이런식 또는,
# 18:00 - xx:00 이런식이다. 이 수업은 실험수업일 것이다.
# 데이터를 가공하기 위해서, 하나의 양식으로 통일해줄 필요가 있다.

for c in final_classrooms:
  print(c)

월 10:30(75) 201-6514
수 10:30(75) 201-6514
월 10:30(75) 201-6516
수 10:30(75) 201-6516
화 13:30(75) 201-6202
목 13:30(75) 201-6202
화 13:30(75) 201-6203
목 13:30(75) 201-6203
화 10:30(75) 201-6514
목 10:30(75) 201-6514
월 16:30(75) 201-6516
수 16:30(75) 201-6516
화 09:00(75) 201-6516
목 09:00(75) 201-6516
화 15:00(75) 201-6514
목 15:00(75) 201-6514
월 09:00(75) 201-6514
수 09:00(75) 201-6514
월 16:30(75) 201-6203
수 16:30(75) 201-6203
화 16:30(75) 201-6202
목 16:30(75) 201-6202
화 13:30(75) 201-6514
목 13:30(75) 201-6514
화 10:30(75) 201-6202
목 10:30(75) 201-6202
화 15:00(75) 201-6202
목 15:00(75) 201-6202
화 16:30(100) 201-6409
목 16:30(100) 201-6409
월 10:30(100) 201-6409-1
수 10:30(100) 201-6409-1
월 16:30(100) 201-6308-1
수 16:30(100) 201-6308-1
화 10:30(100) 201-6308-1
목 10:30(100) 201-6308-1
화 15:00(75) 201-6515
목 15:00(75) 201-6515
화 09:00(75) 201-6514
목 09:00(75) 201-6514
월 16:30(75) 201-6514
수 16:30(75) 201-6514
월 13:30(75) 201-6202
수 13:30(75) 201-6202
월 15:00(75) 201-6515
수 15:00(75) 201-6515
화 09:00(75) 20

In [60]:
# 평소에는 쓰지 않는 정규표현식을 적극적으로 활용해보겠다. (정규표현식으로 하면 코드가 깔끔해질것 같기 때문이다)
# 실험 강의 데이터 'xx:00-xx:00'를 'xx:00(xx)' 형식으로 가공해주겠다.

import re
for i in range(len(final_classrooms)):
  type1 = re.findall('[0-9]+:[0-9]+-[0-9]+:[0-9]+', final_classrooms[i])
  if type1:
    type1 = type1[0]
    start_time, end_time = type1.split('-')[0], type1.split('-')[1]
    start_minute = int(start_time.split(':')[0]) * 60 + int(start_time.split(':')[1])
    end_minute = int(end_time.split(':')[0]) * 60 + int(end_time.split(':')[1])
    running_time = end_minute - start_minute

    final_classrooms[i] = final_classrooms[i].replace(type1, start_time + '(' + str(running_time) + ')')

월 18:00(180) 201-6409-1
수 18:00(180) 201-6409-1
월 18:00(240) 201-6408
화 18:00(240) 201-6408
수 18:00(240) 201-6408


In [14]:
# 총 107개의 수업이 있다! 물론 비대면도 포함된 수업들이고, 교수계획표를 작성하지 않은 교수님도 있기 때문에 확실하진 않다.
print(len(final_classrooms))

107


In [39]:
# 우리 과 건물번호는 201번이다. 건물은 1층부터 5층까지 이루어져 있지만, 
# 이중에는 수업을 안하는 강의실과, 다른 과에서 수업하는 강의실도 있다.
# 그러므로, 우리과에서만 수업을 진행하는 강의실을 살펴보기로 하자.


classroom_set = set()
for classroom_information in final_classrooms:
  room = re.findall('201-\d*', classroom_information)[0][4:]
  classroom_set.add(room)

In [40]:
# 이용하는 강의실은 총 8개이다!

print(classroom_set)

{'6516', '6515', '6202', '6308', '6408', '6203', '6514', '6409'}


In [208]:
# 수업 시간은 75분, 100분, 180분, 240분 이렇게 이루어져 있으므로, 최대공약수가 25분이다.
# 즉, 오전 9시부터 오후 10시까지 25분 단위로 모든 강의실의 시간들을 초기화 시켜준다.

total_information = []


for i in range(5):
  classroom_dict = {room : [] for room in classroom_set}
  for room in classroom_dict:
    for tim in range(9*60, 22*60):  
      classroom_dict[room].append(tim)
  
  total_information.append(classroom_dict)

In [210]:
days = {'월' : 0, "화" : 1, "수" : 2, "목" : 3, "금" : 4}

for classroom_information in final_classrooms:
  day, tim, location = classroom_information.split()
  day = days[day]
  start_time, running_time = tim.split('(')[0], int(tim.split('(')[1][:-1])
  start_time = int(start_time.split(':')[0]) * 60 + int(start_time.split(':')[1])

  location = location.split('-')[1]

  for tmp in range(0, running_time):
    try:
      total_information[day][location].remove(start_time + tmp)
    except:
      continue

In [212]:
for d in range(5):
  for room in classroom_set:
    new_time = []
    timelist = total_information[d][room]
    start_time = timelist[0]
    for i in range(1, len(timelist)):
      if timelist[i] - timelist[i-1] > 1:
        end_time = timelist[i-1]

        if end_time - start_time >= 30:
          new_time.append(str(start_time//60).zfill(2) + ':' + str(start_time%60).zfill(2) + '-' + str(end_time//60).zfill(2) + ':' + str(end_time%60).zfill(2))
        start_time = timelist[i]
    
    new_time.append(str(start_time//60).zfill(2) + ':' + str(start_time%60).zfill(2) + '-' + '22:00')
    total_information[d][room] = new_time

In [215]:
import pandas as pd

In [216]:
pd.DataFrame(total_information, index = ["월", "화", "수", "목", "금"])

,6516,6515,6202,6308,6408,6203,6514,6409
월,"[09:00-10:29, 11:45-13:29, 17:45-22:00]","[09:00-13:29, 16:15-22:00]","[09:00-13:29, 16:15-22:00]","[09:00-10:29, 12:10-16:29, 18:10-22:00]",[12:10-22:00],"[09:00-13:29, 17:45-22:00]","[11:45-13:29, 17:45-22:00]","[12:10-16:29, 21:00-22:00]"
화,"[11:45-13:29, 17:45-22:00]","[09:00-14:59, 16:15-22:00]","[11:45-13:29, 17:45-22:00]","[09:00-10:29, 12:10-16:29, 18:10-22:00]","[09:00-10:29, 12:10-13:29, 17:45-22:00]","[11:45-13:29, 14:45-22:00]","[11:45-13:29, 16:15-22:00]","[10:15-13:29, 18:10-22:00]"
수,"[09:00-10:29, 11:45-13:29, 17:45-22:00]","[09:00-13:29, 16:15-22:00]","[09:00-13:29, 16:15-22:00]","[09:00-10:29, 12:10-16:29, 18:10-22:00]",[12:10-22:00],"[09:00-13:29, 17:45-22:00]","[11:45-13:29, 17:45-22:00]","[12:10-16:29, 21:00-22:00]"
목,"[11:45-13:29, 17:45-22:00]","[09:00-14:59, 16:15-22:00]","[11:45-13:29, 17:45-22:00]","[09:00-10:29, 12:10-16:29, 18:10-22:00]","[09:00-10:29, 12:10-13:29, 17:45-22:00]","[11:45-13:29, 14:45-22:00]","[11:45-13:29, 16:15-22:00]","[10:15-13:29, 18:10-22:00]"
금,[09:00-22:00],[09:00-22:00],[09:00-22:00],[09:00-22:00],[09:00-22:00],[09:00-22:00],[09:00-22:00],[09:00-22:00]


In [162]:
# 근데 결과를 보고 생각해보면, 비대면 수업들이 많아서, 딱히 의미가 없는것 같긴하다....
# 지난주 수요일 6202에서 4시반에 공부 했었기 때문에...
# 그래도 어느정도 후보를 정할 수 있는 것에 만족하자!